In [3]:
import pandas as pd
import requests
import warnings
from functools import reduce

warnings.filterwarnings('ignore')

In [4]:
mfs = {
    "largecap": (120586, 20),  #ICICI Prudential Bluechip Fund
    "midcap": (118989, 9),  #HDFC Mid-Cap Opportunities Fund
    "smallcap": (118778, 21),  #Nippon India Small Cap Fund
    "liquid": (119800, 22),  #SBI Liquid Fund
    "gold": (119132, 9),  #HDFC Gold Fund
}

In [5]:
dfs = []
for mf in mfs:
    mfId = mfs[mf][1]
    scId = mfs[mf][0]
    payloads = [{
        'mfID': mfId,
        'scID': scId,
        'fDate': f'01-Jan-{i}',
        'tDate': f'31-Dec-{min(i + 4, 2023)}',
    } for i in range(2010, 2023, 5)]

    df = pd.DataFrame(columns=[mf, 'date'])
    for payload in payloads:
        response = requests.post('https://www.amfiindia.com/modules/NavHistoryPeriod', data=payload)
        df_part = pd.read_html(response.text)[0]
        df_part = df_part[[df_part.columns[0], df_part.columns[-1]]]
        df_part.columns = [mf, 'date']
        df = pd.concat([df, df_part])
    df = df[df.columns[::-1]]
    dfs.append(df)

KeyboardInterrupt: 

In [55]:
df = reduce(lambda l,r: pd.merge(l,r,left_on='date',right_on='date',how='inner'), dfs)

In [57]:
df.to_csv("data/mfs.csv",index=False)